## Random Graphs

### Helper Functions

Randomly generate an adjancency matrix...

In [1]:
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, groupby

n = 4

def random_row(row_num):
    row = list(np.random.choice((0, 1, 2, 3, 4, 5),size=n))
    
    # No self-references
    row[row_num] = 0 
    
    # Make sure there's no isolated nodes
    if sum(row) == 0:
        row = random_row(1)
    return row

def random_matrix():
    rows = []
    for i in range(0, n):
        rows.append(random_row(1))
    numpy_matrix = np.matrix(rows)
    
    return numpy_matrix

def random_graph():
    np_matrix = random_matrix()
    g = nx.convert_matrix.from_numpy_matrix(
        np_matrix, 
        parallel_edges=False, 
        create_using=nx.Graph
    )
    
    # nx.draw(g, node_color='lightblue', 
    #     with_labels=True, 
    #     node_size=500)
    
    return g, np_matrix

### Brute Force

In [2]:
def brute_force(graph, w):

    best_cost_brute = 0
    for b in range(2**n):
        x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
        cost = 0
        for i in range(n):
            for j in range(n):
                cost = cost + w[i,j]*x[i]*(1-x[j])
        if best_cost_brute < cost:
            best_cost_brute = cost
            xbest_brute = x
        #print('case = ' + str(x)+ ' cost = ' + str(cost))

    return xbest_brute, best_cost_brute

### Variatonal Quantum EignSolver

In [3]:
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer

def vqe(w):
    
    # Define our Qiskit Maxcut Instance
    max_cut = Maxcut(w)
    qp = max_cut.to_quadratic_program()

    # Translate to Ising Hamiltonian
    qubitOp, offset = qp.to_ising()

    # Setup our simulator
    algorithm_globals.random_seed = 123
    seed = 10598
    backend = Aer.get_backend('aer_simulator_statevector')
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

    # Construct VQE
    spsa = SPSA(maxiter=300)
    cobyla = COBYLA(maxiter=300)
    ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    vqe = VQE(ry, optimizer=spsa, quantum_instance=quantum_instance)

    # Run VQE
    result = vqe.compute_minimum_eigenvalue(qubitOp)
    print(result)

    # print results
    bit_string = list(max_cut.sample_most_likely(result.eigenstate))
    bit_string = map(int, bit_string)
    # print('energy:', result.eigenvalue.real)
    # print('time:', result.optimizer_time)
    # print('max-cut objective:', result.eigenvalue.real + offset)
    # print('solution:', bit_string)
    # print('solution objective:', qp.objective.evaluate(bit_string))
    
    return bit_string, (result.eigenvalue.real + offset), result.optimizer_time

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
# Generate 1,000 graphs
success = 0
failure = 0

# TODO: Change the number of tests, ad hoc
num_tests: int = 1

for graph_id in range(num_tests):
    # Generate Random Graph
    graph, matrix = random_graph()
    
    # Computing the weight matrix from the random graph
    w = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            temp = graph.get_edge_data(i,j,default=0)
            if temp != 0:
                w[i,j] = temp['weight']
    
    bf_bit_string, bf_cost = brute_force(graph, w)
    bf_x = ''.join(str(e) for e in bf_bit_string)
    bf_x_inverse = ''.join('1' if x == '0' else '0' for x in bf_x)
    print(f'\nBrute Force: Best solution = {bf_x} | inverse[{bf_x_inverse}]  cost = {str(bf_cost)}')
    
    vqe_bit_string, vqe_cost, vqe_time = vqe(w)
    vqe_x = ''.join(str(e) for e in vqe_bit_string)
    vqe_x_inverse = ''.join('1' if x == '0' else '0' for x in vqe_x)
    print(f'VQE: Best solution = {vqe_x} | inverse[{vqe_x_inverse}]  cost = {str(vqe_cost)}  {vqe_time}')
    print()
    print(w)
    print()
    print()
    
    # Check validity
    if bf_x == vqe_x or bf_x == vqe_x_inverse or vqe_x == bf_x_inverse:
        success += 1
    else:
        failure += 1


Brute Force: Best solution = 1100 | inverse[0011]  cost = 12.0
{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 600,
    'eigenstate': array([-3.17146395e-04+0.j,  3.07747588e-03+0.j, -2.82457451e-04+0.j,
       -1.46665326e-05+0.j,  1.15264494e-04+0.j,  3.35095381e-03+0.j,
        6.46841977e-02+0.j, -1.65962598e-03+0.j,  3.27670158e-03+0.j,
        9.97759353e-01+0.j, -8.11917908e-05+0.j, -1.60613267e-03+0.j,
       -1.17624289e-05+0.j,  3.58356653e-03+0.j, -1.48538520e-02+0.j,
       -4.66679535e-03+0.j]),
    'eigenvalue': (-4.998351993459096+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[0]): 2.98453641248967,
                              ParameterVectorElement(θ[2]): -3.840731778514316,
                              ParameterVectorElement(θ[3]): -5.108757876702927,
                              ParameterVectorElement(θ[4]): -7.0816450605871735,
                              ParameterVectorElement(θ[5]): 6.14826492695333,
                            

In [5]:
print(f"Success = {success}")
print(f"Failures = {failure}")

Success = 0
Failures = 1
